## Investigate a nc file

In [ ]:
import xarray as xr

ds = xr.open_dataset("C:/Users/15311/Desktop/CEDS_2017/ALD2-em-total-anthro_CEDS_2017.nc")
df = ds.to_dataframe()

In [ ]:
df

## Process nc files

In [1]:
import os

path = "C:/Users/15311/Desktop/CEDS_2017"
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f[-3:]==".nc"]

In [3]:
files.remove('CEDS_gridcell_area_05.nc')

In [4]:
'CEDS_gridcell_area_05.nc' in files

False

In [30]:
# for pol in pols:        
#     f = '%s-em-total-anthro_CEDS_2017.nc'%pol
#     print(os.path.join(path, f).replace('\\','/'))

In [5]:
import netCDF4

# Get area of grid cells
area = netCDF4.Dataset("C:/Users/15311/Desktop/CEDS_2017/CEDS_gridcell_area_05.nc", "r").variables["gridcell area"][:] # m^2

sectors = ["agr", "ene", "ind", "nrtr","rcoc", "rcoo", "rcor", "road", "shp", "slv", "wst"]
pols = [p.split("-")[0] for p in files]

print(pols)

vocs = ['ALD2', 'ALK4_butanes', 'ALK4_hexanes', 'ALK4_pentanes', 'BENZ', 'BUTENE', 
        'C2H2', 'C2H4', 'C2H6', 'C3H8', 'CH2O', 'CHC', 'EOH', 'ESTERS', 'ETHERS', 
        'MEK', 'OTHER_AROM', 'OTHER_VOC', 'PRPE', 'TMB', 'TOLU', 'XYLE', 'HCOOH']
pm = ['BC', 'OC'] # I think we can assume both BC and OC are pm2.5?

noToNOx = 1.0 / 0.65 # Convert NO-based mass to NO2-based mass (the opposite of what is done in CEDS_edit.sh)

# Emissions units = kg m-2 s-1
secondsPerYear = 60 * 60 * 24 * 365 + 1 # 2017 year second (2016 was a leap year with a leap second.)

dims = {}

for sector in sectors:
    out = netCDF4.Dataset('ceds_%s.nc'%sector, 'w', format='NETCDF3_64BIT_OFFSET', clobber=True)
    for pol in pols:
        if pol in vocs: outpol = "VOC"
        elif pol in pm: outpol = "PM2_5"
        elif pol == "NO": outpol = "NOx"
        elif pol == "SO2": outpol = "SOx"
        else: outpol = pol
        
        f = '%s-em-total-anthro_CEDS_2017.nc'%pol # total anthropogenic emissions "previous file: -em-anthro_CMIP_CEDS_2017.nc"
        filepath = os.path.join(path, f).replace('\\','/') # file path
        rootgrp = netCDF4.Dataset(filepath, "r")
        
        if len(dims) == 0:
            for d in ["lat", "lon"]: dims[d] = rootgrp.dimensions[d]
        
        v = '%s_%s'%(pol, sector)
        data = rootgrp.variables[v][:].mean(axis=0) # remove time dimension
        data = data * secondsPerYear * area # New units: kg year-1
        if pol == "NO": data = data * noToNOx
        
        s = data.sum()
        if s==0.0: continue
        print(sector, pol, outpol, data.sum(), "kg year-1")
        
        if len(out.dimensions) == 0:
            for d in dims: out.createDimension(d,size=dims[d].size)
            for vname in ["lat", "lon"]:
                rootvar = rootgrp.variables[vname]
                x = out.createVariable(vname, rootvar.datatype, rootvar.dimensions)
                x[:] = rootvar[:]
                x.setncatts(rootvar.__dict__)
        
        if outpol not in out.variables.keys(): 
            vout = out.createVariable(outpol, 'f8', rootgrp.variables[v].dimensions[1:])
            vout.units = "kg year-1"
            vout.long_name = rootgrp.variables[v].long_name.replace(pol, outpol)
            out.variables[outpol][:] = data
        else: out.variables[outpol][:] = out.variables[outpol][:] + data
    out.close()

['ALD2', 'ALD2', 'ALD2', 'ALD2', 'ALD2', 'ALK4_butanes', 'ALK4_butanes', 'ALK4_butanes', 'ALK4_butanes', 'ALK4_butanes', 'ALK4_hexanes', 'ALK4_hexanes', 'ALK4_hexanes', 'ALK4_hexanes', 'ALK4_hexanes', 'ALK4_pentanes', 'ALK4_pentanes', 'ALK4_pentanes', 'ALK4_pentanes', 'ALK4_pentanes', 'BC', 'BC', 'BC', 'BC', 'BC', 'BENZ', 'BENZ', 'BENZ', 'BENZ', 'BENZ', 'BUTENE', 'BUTENE', 'BUTENE', 'BUTENE', 'BUTENE', 'C2H2', 'C2H2', 'C2H2', 'C2H2', 'C2H2', 'C2H4', 'C2H4', 'C2H4', 'C2H4', 'C2H4', 'C2H6', 'C2H6', 'C2H6', 'C2H6', 'C2H6', 'C3H8', 'C3H8', 'C3H8', 'C3H8', 'C3H8', 'CH2O', 'CH2O', 'CH2O', 'CH2O', 'CH2O', 'CHC', 'CHC', 'CHC', 'CHC', 'CHC', 'CO', 'CO', 'CO', 'CO', 'CO', 'EOH', 'EOH', 'EOH', 'EOH', 'EOH', 'ESTERS', 'ESTERS', 'ESTERS', 'ESTERS', 'ESTERS', 'ETHERS', 'ETHERS', 'ETHERS', 'ETHERS', 'ETHERS', 'HCOOH', 'HCOOH', 'HCOOH', 'HCOOH', 'HCOOH', 'MEK', 'MEK', 'MEK', 'MEK', 'MEK', 'NH3', 'NH3', 'NH3', 'NH3', 'NH3', 'NO', 'NO', 'NO', 'NO', 'NO', 'OC', 'OC', 'OC', 'OC', 'OC', 'OTHER_AROM', 'OTHE

C:\Users\15311\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:54: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
C:\Users\15311\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


agr NH3 NH3 45550213554.15599 kg year-1
agr NH3 NH3 45550213554.15599 kg year-1
agr NO NOx 5701241779.330917 kg year-1
agr NO NOx 5701241779.330917 kg year-1
agr NO NOx 5701241779.330917 kg year-1
agr NO NOx 5701241779.330917 kg year-1
agr NO NOx 5701241779.330917 kg year-1
ene ALD2 VOC 92387304.92163344 kg year-1
ene ALD2 VOC 92387304.92163344 kg year-1
ene ALD2 VOC 92387304.92163344 kg year-1
ene ALD2 VOC 92387304.92163344 kg year-1
ene ALD2 VOC 92387304.92163344 kg year-1
ene ALK4_butanes VOC 14427744414.640142 kg year-1
ene ALK4_butanes VOC 14427744414.640142 kg year-1
ene ALK4_butanes VOC 14427744414.640142 kg year-1
ene ALK4_butanes VOC 14427744414.640142 kg year-1
ene ALK4_butanes VOC 14427744414.640142 kg year-1
ene ALK4_hexanes VOC 7887520192.610528 kg year-1
ene ALK4_hexanes VOC 7887520192.610528 kg year-1
ene ALK4_hexanes VOC 7887520192.610528 kg year-1
ene ALK4_hexanes VOC 7887520192.610528 kg year-1
ene ALK4_hexanes VOC 7887520192.610528 kg year-1
ene ALK4_pentanes VOC 171

ind CH2O VOC 308339773.8044384 kg year-1
ind CH2O VOC 308339773.8044384 kg year-1
ind CH2O VOC 308339773.8044384 kg year-1
ind CH2O VOC 308339773.8044384 kg year-1
ind CO CO 76873522159.11499 kg year-1
ind CO CO 76873522159.11499 kg year-1
ind CO CO 76873522159.11499 kg year-1
ind CO CO 76873522159.11499 kg year-1
ind CO CO 76873522159.11499 kg year-1
ind EOH VOC 127184697.90171121 kg year-1
ind EOH VOC 127184697.90171121 kg year-1
ind EOH VOC 127184697.90171121 kg year-1
ind EOH VOC 127184697.90171121 kg year-1
ind EOH VOC 127184697.90171121 kg year-1
ind HCOOH VOC 496241390.7941995 kg year-1
ind HCOOH VOC 496241390.7941995 kg year-1
ind HCOOH VOC 496241390.7941995 kg year-1
ind HCOOH VOC 496241390.7941995 kg year-1
ind HCOOH VOC 496241390.7941995 kg year-1
ind MEK VOC 40609683.772951156 kg year-1
ind MEK VOC 40609683.772951156 kg year-1
ind MEK VOC 40609683.772951156 kg year-1
ind MEK VOC 40609683.772951156 kg year-1
ind MEK VOC 40609683.772951156 kg year-1
ind NH3 NH3 1193774077.252

rcoc ALD2 VOC 12247207.432142569 kg year-1
rcoc ALK4_butanes VOC 8017686.04300126 kg year-1
rcoc ALK4_butanes VOC 8017686.04300126 kg year-1
rcoc ALK4_butanes VOC 8017686.04300126 kg year-1
rcoc ALK4_butanes VOC 8017686.04300126 kg year-1
rcoc ALK4_butanes VOC 8017686.04300126 kg year-1
rcoc ALK4_hexanes VOC 5497836.506583672 kg year-1
rcoc ALK4_hexanes VOC 5497836.506583672 kg year-1
rcoc ALK4_hexanes VOC 5497836.506583672 kg year-1
rcoc ALK4_hexanes VOC 5497836.506583672 kg year-1
rcoc ALK4_hexanes VOC 5497836.506583672 kg year-1
rcoc ALK4_pentanes VOC 11366400.252219032 kg year-1
rcoc ALK4_pentanes VOC 11366400.252219032 kg year-1
rcoc ALK4_pentanes VOC 11366400.252219032 kg year-1
rcoc ALK4_pentanes VOC 11366400.252219032 kg year-1
rcoc ALK4_pentanes VOC 11366400.252219032 kg year-1
rcoc BC PM2_5 265008579.47879314 kg year-1
rcoc BC PM2_5 265008579.47879314 kg year-1
rcoc BC PM2_5 265008579.47879314 kg year-1
rcoc BC PM2_5 265008579.47879314 kg year-1
rcoc BC PM2_5 265008579.478793

rcoo CH2O VOC 28424416.260211892 kg year-1
rcoo CH2O VOC 28424416.260211892 kg year-1
rcoo CH2O VOC 28424416.260211892 kg year-1
rcoo CH2O VOC 28424416.260211892 kg year-1
rcoo CO CO 15110836509.250292 kg year-1
rcoo CO CO 15110836509.250292 kg year-1
rcoo CO CO 15110836509.250292 kg year-1
rcoo CO CO 15110836509.250292 kg year-1
rcoo CO CO 15110836509.250292 kg year-1
rcoo EOH VOC 147180754.82263148 kg year-1
rcoo EOH VOC 147180754.82263148 kg year-1
rcoo EOH VOC 147180754.82263148 kg year-1
rcoo EOH VOC 147180754.82263148 kg year-1
rcoo EOH VOC 147180754.82263148 kg year-1
rcoo ETHERS VOC 74715670.59790377 kg year-1
rcoo ETHERS VOC 74715670.59790377 kg year-1
rcoo ETHERS VOC 74715670.59790377 kg year-1
rcoo ETHERS VOC 74715670.59790377 kg year-1
rcoo ETHERS VOC 74715670.59790377 kg year-1
rcoo HCOOH VOC 367502820.1740725 kg year-1
rcoo HCOOH VOC 367502820.1740725 kg year-1
rcoo HCOOH VOC 367502820.1740725 kg year-1
rcoo HCOOH VOC 367502820.1740725 kg year-1
rcoo HCOOH VOC 367502820.1

rcor PRPE VOC 2425502447.580623 kg year-1
rcor PRPE VOC 2425502447.580623 kg year-1
rcor PRPE VOC 2425502447.580623 kg year-1
rcor PRPE VOC 2425502447.580623 kg year-1
rcor SO2 SOx 2982594335.8264713 kg year-1
rcor SO2 SOx 2982594335.8264713 kg year-1
rcor SO2 SOx 2982594335.8264713 kg year-1
rcor SO2 SOx 2982594335.8264713 kg year-1
rcor SO2 SOx 2982594335.8264713 kg year-1
rcor TMB VOC 647570.3190347082 kg year-1
rcor TMB VOC 647570.3190347082 kg year-1
rcor TMB VOC 647570.3190347082 kg year-1
rcor TMB VOC 647570.3190347082 kg year-1
rcor TMB VOC 647570.3190347082 kg year-1
rcor TOLU VOC 1426188055.5319211 kg year-1
rcor TOLU VOC 1426188055.5319211 kg year-1
rcor TOLU VOC 1426188055.5319211 kg year-1
rcor TOLU VOC 1426188055.5319211 kg year-1
rcor TOLU VOC 1426188055.5319211 kg year-1
rcor XYLE VOC 430061146.6906542 kg year-1
rcor XYLE VOC 430061146.6906542 kg year-1
rcor XYLE VOC 430061146.6906542 kg year-1
rcor XYLE VOC 430061146.6906542 kg year-1
rcor XYLE VOC 430061146.6906542 kg

shp CO CO 758721144.3425981 kg year-1
shp CO CO 758721144.3425981 kg year-1
shp CO CO 758721144.3425981 kg year-1
shp CO CO 758721144.3425981 kg year-1
shp CO CO 758721144.3425981 kg year-1
shp NH3 NH3 17270938.20458973 kg year-1
shp NH3 NH3 17270938.20458973 kg year-1
shp NH3 NH3 17270938.20458973 kg year-1
shp NH3 NH3 17270938.20458973 kg year-1
shp NH3 NH3 17270938.20458973 kg year-1
shp NO NOx 24933576211.981827 kg year-1
shp NO NOx 24933576211.981827 kg year-1
shp NO NOx 24933576211.981827 kg year-1
shp NO NOx 24933576211.981827 kg year-1
shp NO NOx 24933576211.981827 kg year-1
shp OC PM2_5 139606769.73980018 kg year-1
shp OC PM2_5 139606769.73980018 kg year-1
shp OC PM2_5 139606769.73980018 kg year-1
shp OC PM2_5 139606769.73980018 kg year-1
shp OC PM2_5 139606769.73980018 kg year-1
shp OTHER_AROM VOC 175425073.4134046 kg year-1
shp OTHER_AROM VOC 175425073.4134046 kg year-1
shp OTHER_AROM VOC 175425073.4134046 kg year-1
shp OTHER_AROM VOC 175425073.4134046 kg year-1
shp OTHER_AR

wst HCOOH VOC 145722820.78024516 kg year-1
wst HCOOH VOC 145722820.78024516 kg year-1
wst HCOOH VOC 145722820.78024516 kg year-1
wst HCOOH VOC 145722820.78024516 kg year-1
wst HCOOH VOC 145722820.78024516 kg year-1
wst MEK VOC 17508505.38732829 kg year-1
wst MEK VOC 17508505.38732829 kg year-1
wst MEK VOC 17508505.38732829 kg year-1
wst MEK VOC 17508505.38732829 kg year-1
wst MEK VOC 17508505.38732829 kg year-1
wst NH3 NH3 8639312757.412424 kg year-1
wst NH3 NH3 8639312757.412424 kg year-1
wst NH3 NH3 8639312757.412424 kg year-1
wst NH3 NH3 8639312757.412424 kg year-1
wst NH3 NH3 8639312757.412424 kg year-1
wst NO NOx 2469988449.3190923 kg year-1
wst NO NOx 2469988449.3190923 kg year-1
wst NO NOx 2469988449.3190923 kg year-1
wst NO NOx 2469988449.3190923 kg year-1
wst NO NOx 2469988449.3190923 kg year-1
wst OC PM2_5 1706103511.3703773 kg year-1
wst OC PM2_5 1706103511.3703773 kg year-1
wst OC PM2_5 1706103511.3703773 kg year-1
wst OC PM2_5 1706103511.3703773 kg year-1
wst OC PM2_5 1706